In [ ]:
import sys
sys.path.append("../")


import numpy as np
name = 's13to8'
train = np.loadtxt( name + ".ts.data", dtype=float, delimiter=",", skiprows=0)
test = np.loadtxt( name + ".test.data", dtype=float, delimiter=",", skiprows=0)

D = np.vstack((train, test))
F = D.shape[1]
features = ["V" + str(i) for i in range(F)]
name.upper()

from spn.structure.Base import Context
from spn.algorithms.LearningWrappers import learn_parametric
from spn.structure.leaves.parametric.Parametric import Bernoulli

try:
    from time import perf_counter
except:
    from time import time as perf_counter
    
    
np.seterr(divide='ignore', invalid='ignore')
ds_context = Context(parametric_types= np.asarray([Bernoulli] * F)).add_domains(train)
eval_start_t = perf_counter()
spn = learn_parametric(train, ds_context, min_instances_slice=20)
eval_end_t = perf_counter()
#print('Structure of SPN learnt in {0} secs'.format(eval_end_t - eval_start_t))


'''
from spn.algorithms.Statistics import get_structure_stats
print(get_structure_stats(spn))
print('\n')

from spn.io.Graphics import *
def plot_spn_svg(root_node, fname="spn.pdf"):
    import networkx.drawing.nx_pydot as nxpd

    g, _ = get_networkx_obj(root_node)

    pdG = nxpd.to_pydot(g)
    svg_string = pdG.create_svg()

    f = open(fname, 'wb')
    f.write(svg_string)

def plot(spn, fname="spn.pdf"):

    import networkx as nx
    from networkx.drawing.nx_pydot import graphviz_layout

    import matplotlib.pyplot as plt

    plt.clf()

    g, labels = get_networkx_obj(spn)

    pos = graphviz_layout(g, prog='dot')
    #plt.figure(figsize=(200, 100))
    ax = plt.gca()

    ax.invert_yaxis()

    nx.draw(g, pos, with_labels=True, arrows=False, node_color='blue', edge_color='black', width=1, node_size=1,
            labels=labels, font_size=1)
    ax.collections[0].set_edgecolor("#333333")
    #edge_labels = nx.draw_networkx_edge_labels(g, pos=pos, edge_labels=nx.get_edge_attributes(g, 'weight'), font_size=1,
                                              # clip_on=False, alpha=0.6)

    xpos = list(map(lambda p: p[0], pos.values()))
    ypos = list(map(lambda p: p[1], pos.values()))

    ax.set_xlim(min(xpos)-20,max(xpos)+20)
    ax.set_ylim(min(ypos)-20,max(ypos)+20)
    plt.tight_layout()
    plt.margins(0, 0)
    plt.gca().xaxis.set_major_locator(NullLocator())
    plt.gca().yaxis.set_major_locator(NullLocator())
    plt.savefig(fname,bbox_inches='tight', pad_inches = 0)
plot(spn)
plot_spn_svg(spn)
'''
print("............................................\n\n\n\n\n\n")
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import pearsonr
from spn.algorithms.Sampling import sample_instances
from numpy.random.mtrand import RandomState
from copy import deepcopy

# test[:,0:5] : These are the galaxy properties to be predicted

test_copy = deepcopy(test)
test_copy[:,0:5] = np.nan

sampled_data = sample_instances(spn, test_copy, RandomState(123))

print('MSE numOfGalaxies is %f,massCentralBlackHole is %f, stellarMass is %f, velDisp is %f,sfr is %f \n' %
  (mean_squared_error(test[: , 0], sampled_data[: , 0]), mean_squared_error(test[: , 1], sampled_data[: , 1]), \
    mean_squared_error(test[: , 2], sampled_data[: , 2]), mean_squared_error(test[: , 3], sampled_data[: , 3]), \
    mean_squared_error(test[: , 4], sampled_data[: , 4])))

print('R^2 numOfGalaxies is %f, massCentralBlackHole is %f,stellarMasse is %f,velDisp is %f, sfr is %f \n' %
  (r2_score(test[: , 0], sampled_data[: , 0]), r2_score(test[: , 1], sampled_data[: , 1]), \
    r2_score(test[: , 2], sampled_data[: , 2]), r2_score(test[: , 3], sampled_data[: , 3]), \
    r2_score(test[: , 4], sampled_data[: , 4])))

print('P_Co numOfGalaxies is %f, massCentralBlackHole is %f, stellarMass is %f,velDisp is %f, sfr is %f\n' %
  (pearsonr(test[: , 0], sampled_data[: , 0])[0], pearsonr(test[: , 1], sampled_data[: , 1])[0], \
    pearsonr(test[: , 2], sampled_data[: , 2])[0], pearsonr(test[: , 3], sampled_data[: , 3])[0], \
    pearsonr(test[: , 4], sampled_data[: , 4])[0]))
      
      
      
      

    

In [ ]:
from copy import deepcopy
from spn.algorithms.Inference import log_likelihood
test_copy = deepcopy(test)
test_copy[:,0:5] = np.nan

conditional_ll = log_likelihood(spn, test) - log_likelihood(spn, test_copy)
conditional_ll

In [ ]:
np.exp(conditional_ll)

In [ ]:
conditional_ll.shape

In [ ]:

for i in range(len(conditional_ll)):
    if np.exp(conditional_ll[i]) >= 0.5:
        counter += 1
        
counter / len(conditional_ll)   
    

In [ ]:
len(conditional_ll)

In [ ]:
np.arange(0, 5, 1)

In [ ]:
from spn.algorithms.Marginalization import marginalize
spn_marg = marginalize(spn, np.arange(5, 67,1))

In [ ]:
from spn.algorithms.Inference import log_likelihood, conditional_log_likelihood

conditional_log = conditional_log_likelihood(spn, spn_marg, test) 


In [ ]:
conditional_log


In [ ]:
assert(np.all(np.isclose(conditional_ll, conditional_log)))

In [ ]:
assert(np.all(np.isclose(np.exp(conditional_ll), np.exp(conditional_log))))

In [ ]:
from spn.algorithms.LearningWrappers import learn_classifier

classify = learn_classifier(train, ds_context,learn_parametric, label_idx= 0)


In [ ]:
t = deepcopy(test)

In [ ]:
t[:,0] = np.nan

In [ ]:
from spn.algorithms.MPE import mpe
c = mpe(classify, t)

In [ ]:
assert(np.all(np.isclose(c, test)))

In [ ]:
d= sam  

In [ ]:
test

In [ ]:
count = 0
for co in range(len(test)):
    if np.all(test[co,0]) == np.all(c[co,0]):
        
        count +=1
        
count/len(test)        

In [ ]:
count

In [ ]:
from spn.algorithms.Sampling import sample_instances
from numpy.random.mtrand import RandomState
d = sample_instances(classify, t, RandomState(123))

cout = 0
for co in range(len(test)):
    if (test[co,0]) == (d[co,0]):
        
        cout +=1
        
cout/len(test)